<a href="https://colab.research.google.com/github/Yuvraj2409/BE-PROJECT/blob/master/VehicleDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install mrcnn

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for mrcnn: filename=mrcnn-0.2-cp36-none-any.whl size=54931 sha256=8ad1dad5288cecd5c9f46ff84fc504f5b8d45cb34afc2e817013a57de5bcf903
  Stored in directory: /root/.cache/pip/wheels/11/ed/28/e550ddc897c04c336b923eae4eb35c9aae993d20ce39d9cc40
Successfully built mrcnn


In [0]:
# plot one photograph and mask
from os import listdir
from xml.etree import ElementTree as ET
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from matplotlib import pyplot
from google.colab.patches import cv2_imshow

In [0]:
#from google.colab import drive
#drive.mount('/gdrive')
%matplotlib

Using matplotlib backend: agg


In [0]:
# class that defines and loads the kangaroo dataset
class VehicleDataset(Dataset):
	# load the dataset definitions
  def load_dataset(self, dataset_dir, is_train=True):
		# define one class
    self.add_class("dataset", 1, "vehicle")
		#define data locations
    images_dir = dataset_dir + '/MVI_20011/'
    annotations_dir = dataset_dir
    listdir(images_dir)
		# find all images
    img=[]
    for filename in listdir(images_dir):
      # extract image id
      image_id = filename[3:-4]
      #print(image_id)
      img.append(image_id)
    img.append('00001')
    img.sort()
    print(img)
    #img = [int(x) for x in img ]
    
    for filename in img:
    # skip bad images
			#if image_id in ['00090']:
		  #	continue
			# skip all images after 150 if we are building the train set
      if is_train and int(image_id) >= 601:
        continue
			# skip all images before 150 if we are building the test/val set
      if not is_train and int(image_id) < 601:
        continue
      img_path = images_dir + filename
      ann_path = dataset_dir + '/MVI_20011.xml'
      #print(ann_path)
			# add to dataset
      self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
    

	# extract bounding boxes from an annotation file
  def extract_boxes(self, filename, image_id):
    tree = ET.parse(filename)
    print(tree)
    root = tree.getroot()
    #print(root)
    #path = '/content/drive/My Drive/ATY'
    #tree = ET.parse(path)
    #print(root.tag)
    #abc = (root[2][0][0][0][0].attrib).values())
    #print(list(abc))

    #print(root[2].tag)
    boxes=list()
    for a in root.findall('frame'):
      final=[]
      for c in a[0].findall('target'):
        b=[]
        for d in c.findall('box'):
          coors = d.attrib.values()
          for i in range(len(coors)):
            e=(float([x for x in coors][i]))  
            b.append(int(e))        
          final.append(b)  
          #print(coors)
      boxes.append(final)
      coors = list(coors)
      width = int(float(coors[2]))
      height = int(float(coors[3]))
    #print(boxes)
    return boxes[int(image_id)], width, height

	# load the masks for an image
  def load_mask(self, image_id):
		# get details of image
    info = self.image_info[image_id]
    print(info)
		# define box file location
    path = info['annotation']
    #print(path)
		# load XML
    boxes, w, h = self.extract_boxes(path,image_id)
    #print(boxes)
		# create one array for all masks, each on a different channel
    masks = zeros([(h), (w), len(boxes)])
		# create masks
    class_ids = list()
    #print(boxes)
    for i in range(len(boxes)):
      box = boxes[i]
      #print(box)
      row_s, row_e = boxes[i][1]-boxes[i][3], boxes[i][1]
      col_s, col_e = boxes[i][0], boxes[i][2] + boxes[i][0]
      masks[row_s:row_e, col_s:col_e, i] = 1
      #print(masks)
      class_ids.append(self.class_names.index('vehicle'))
    
    x1=row_s
    print(x1)
    
    return masks, asarray(class_ids)  

	# load an image reference
  def image_reference(self, image_id):
    info = self.image_info[image_id]
    return info['path']

# train set
train_set = VehicleDataset()
train_set.load_dataset('/content/drive/My Drive/ATY', is_train=True)
train_set.prepare()
# load an image
image_id =0
image = train_set.load_image(image_id)
#image_id = 10
image = train_set.load_image(image_id)
cv2.rectangle(image,(520,290),(520+160,290+170),(0,255,0),2)
cv2_imshow(image)
print(image.shape)
# load image mask
mask, class_ids = train_set.load_mask(image_id)
#print(mask.shape)
# plot image
#cv2_imshow(image)
#plot mask
#pyplot.imshow(mask[:, :, 0],cmap ='gray', alpha=0.1)
#pyplot.show()
#cv2.rectangle(image,(row_s,col_s),(row_e,col_e),(0,255,0),6)

['00001', '00001', '00002', '00004', '00005', '00007', '00008', '00009', '00011', '00013', '00015', '00017', '00019', '00021', '00024', '00028', '00029', '00030', '00031', '00032', '00034', '00035', '00037', '00038', '00039', '00040', '00043', '00044', '00045', '00047', '00048', '00050', '00051', '00052', '00054', '00055', '00056', '00057', '00058', '00060', '00061', '00064', '00066', '00067', '00068', '00069', '00072', '00073', '00075', '00076', '00077', '00078', '00079', '00082', '00086', '00090', '00091', '00092', '00097', '00098', '00102', '00103', '00104', '00105', '00108', '00109', '00111', '00114', '00115', '00116', '00118', '00120', '00121', '00128', '00131', '00132', '00135', '00136', '00139', '00142', '00143', '00144', '00145', '00148', '00151', '00152', '00153', '00156', '00157', '00160', '00163', '00164', '00168', '00170', '00171', '00172', '00174', '00175', '00176', '00177', '00178', '00180', '00181', '00182', '00185', '00190', '00191', '00193', '00194', '00196', '00198', 

FileNotFoundError: ignored

In [0]:
import cv2

In [0]:
cv2.rectangle(image,(row_s,col_s),(row_e,col_e),(0,255,0),6)